<a href="https://colab.research.google.com/github/FelidasNet/free-colab-rdp/blob/main/felidas-rdp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab RDP** : Colab Üzerinden Uzak Masaüstü Kullanmak

> **Uyarı : Kripto Madenciliğine Uygun Değildir!<br></br>** 
>**T4 GPU'lar gibi donanım kaynakları neden benim için mevcut değil?** BT mevcut donanımı, uzun süreli hesaplamalar yerine Colaboratory'u etkileşimli olarak kullanan kullanıcılar için önceliklidir. Uzun süreli hesaplamalar için Colaboratory kullanan kullanıcılar, kendilerine sağlanan donanım türü ve / veya donanımın kullanılabileceği süre ile geçici olarak sınırlandırılabilir. Yerel bir çalışma zamanı olan Colaboratory’un kullanıcı arayüzünü kullanmak için yüksek hesaplama ihtiyacı olan kullanıcıları cesaretlendiriyoruz. Kripto madencilik için Colaboratory kullanmanın günlük olarak yasaklandığını ve Colab'ı kullanmanın tamamen yasaklanmasına neden olabileceğini lütfen unutmayın.

Google Colab, Ücretsiz kullanıcılar için 12 saat boyunca (Maks.) 12 GB RAM ve GPU ile Örnek verebilir. Herkes Ağır Görevleri gerçekleştirmek için kullanabilir.

Lütfen topluluğumuzdaki gönderiye girerek bize destekte bulunun: **[Bedava RDP/VPS Nasıl Alınır? 2021 Ücretsiz RDP/VDS/VPS Almak](https://felidas.net/d/283)**

In [ ]:
#@title **Kullanıcı Oluştur**
#@markdown Kullanıcı Adı ve Şifre Girin

import os

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Kullanıcı oluşturuluyor...")

# Creation of user
os.system(f"useradd -m {username}")

# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print("Kullanıcı başarıyla oluşturuldu!")

In [ ]:
#@title **RDP**
#@markdown  Kurulum yaklaşık 4-5 dakika sürebilir

import os
import subprocess

#@markdown Lütfen http://remotedesktop.google.com/headless adresini ziyaret edin ve oturum açtıktan sonra çıkan kodu kopyalayın

CRP = "" #@param {type:"string"}

#@markdown Lütfen 6 ya da daha fazla sayıdan oluşan bir PIN girin
Pin = 123456 #@param {type: "integer"}


class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish()

    @staticmethod
    def installCRD():
        print("Chrome Uzak Masaüstü Kuruluyor")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Masaüstü Ortamı Kuruluyor")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Google Chrome Kuruluyor")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish():
        print("Sonlandırılıyor")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Başarıyla Sonlandırıldı")


try:
    if username:
        if CRP == "":
            print("Lütfen verilen bağlantıdan kimlik doğrulama kodunu girin")
        elif len(str(Pin)) < 6:
            print("Lütfen 6 ya da daha fazla sayıdan oluşan bir PIN girin")
        else:
            CRD()
except NameError as e:
    print("Kullanıcı adı bulunamadı")
    print("Öncelikle kullanıcı oluşturun")

In [ ]:
#@title **Google Drive Bağlantısı**
#@markdown Google Drive dosyalar için HDD görevi görecektir.<br>

def MountGDrive():
    from google.colab import drive

    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

    mount = """from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')""".format(mountpoint)

    with open('/content/mount.py', 'w') as script:
        script.write(mount)

    ! runuser -l $user -c "python3 /content/mount.py"

try:
    if username:
        mountpoint = "/home/"+username+"/drive"
        user = username
except NameError:
    print("Kullanıcı adı bulunamadı, `/content/drive' adresine `root' kullanıcısını kullanarak bağlanılıyor.")
    mountpoint = '/content/drive'
    user = 'root'

MountGDrive()

In [ ]:
#@title **Colab Sonlandırma**

#@markdown NGROK Tünelini Kapat
NGROK = False #@param {type:'boolean'}

#@markdown GDrive Bağlantısını Kes
GDrive = False #@param {type:'boolean'}

#@markdown Colab'ı Uyut
Sleep = True #@param {type:'boolean'}

if NGROK:
    ! killall ngrok

if GDrive:
    with open('/content/unmount.py', 'w') as unmount:
        unmount.write("""from google.colab import drive
drive.flush_and_unmount()""")
    
    try:
        if user:
            ! runuser $user -c 'python3 /content/unmount.py'
    except NameError:
        print("Google Drive Bağlanmamış!")

if Sleep:
    from time import sleep
    sleep(43200)